In [ ]:
using Minuit2
using Plots
using Distributions
using FHist
using FiniteDiff: finite_difference_gradient
using BenchmarkTools
BenchmarkTools.DEFAULT_PARAMETERS.samples = 100
using Profile

In [ ]:
# Generate some data
const t_μ, t_σ = 1.0, 0.1
x = rand(Normal(t_μ, t_σ), 2000)

h = Hist1D(x, binedges=range(.5, 1.5, 51))

In [ ]:
cdf_model(x, μ, σ) = cdf(Normal(μ, σ), x)
cdf_grad(x, μ, σ) = finite_difference_gradient(p -> cdf_model(x, p...), [μ, σ])
pdf_model(x, μ, σ) = pdf(Normal(μ, σ), x)
pdf_grad(x, μ, σ) = finite_difference_gradient(p -> pdf_model(x, p...), [μ, σ])

## Binned

In [ ]:
cost = BinnedNLL(h, cdf_model, grad=cdf_grad)
m = Minuit(cost, μ=0., σ=1., limit_σ=(0, Inf))
migrad!(m)
visualize(m)

In [ ]:
@benchmark migrad!(m)

In [ ]:
cost = BinnedNLL(h, cdf_model)
m = Minuit(cost, μ=0., σ=1., limit_σ=(0, Inf))
migrad!(m)
visualize(m)

In [ ]:
@benchmark migrad!(m)

## Unbinned

In [ ]:
cost = UnbinnedNLL(x, pdf_model, grad=pdf_grad)
m = Minuit(cost, μ=0., σ=1., limit_σ=(0, Inf))
migrad!(m)
visualize(m)

In [ ]:
@benchmark migrad!(m)

In [ ]:
cost = UnbinnedNLL(x, pdf_model)
m = Minuit(cost, μ=0., σ=1., limit_σ=(0, Inf))
migrad!(m)
visualize(m)

In [ ]:
@benchmark migrad!(m)

In [ ]:
@profile for i in 1:1000
    migrad!(m)
end

In [ ]:
Profile.print()

In [ ]:
@benchmark value(m.cost, [1.,.1])